In [ ]:

import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

import os
no_diabetic_train = os.listdir("/content/drive/MyDrive/Smart Med Tech/ThermoFoot Dataset/train/Control Group")
diabetic_train = os.listdir("/content/drive/MyDrive/Smart Med Tech/ThermoFoot Dataset/train/DM Group")



import cv2
label_train = []
images_train = []
folder_control = "/content/drive/MyDrive/Smart Med Tech/ThermoFoot Dataset/train/Control Group/"
for image in no_diabetic_train:
  path = folder_control+image
  img = cv2.imread(path)
  if img.shape == (224,224,3):
    label_train.append(0)
    images_train.append(img)

folder_dm = "/content/drive/MyDrive/Smart Med Tech/ThermoFoot Dataset/train/DM Group/"
for image in diabetic_train:
  path = folder_dm + image
  img = cv2.imread(path)
  if img.shape == (224,224,3):
    label_train.append(1)
    images_train.append(img)

import random
import numpy as np
temp = list(zip(images_train,label_train))
random.shuffle(temp)
images_train,label_train  = zip(*temp)
images_train = np.asarray([np.asarray(image) for image in images_train])
label_train = np.asarray(label_train)


train_images = images_train.astype('float32') / 255
train_labels = tf.keras.utils.to_categorical(label_train, num_classes=2)




# Load the pre-trained VGG16 model
vgg = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze all layers in VGG16
for layer in vgg.layers:
    layer.trainable = False
# Add custom dense layers on top of VGG16
x = Flatten()(vgg.output)
x = Dense(256, activation='relu')(x)
x = Dense(2, activation='softmax')(x)

# Create the fine-tuned model
model = Model(inputs=vgg.input, outputs=x)

# Unfreeze the last few layers for fine-tuning
for layer in model.layers[-5:]:
    layer.trainable = True

# Define a custom optimizer with a lower learning rate for fine-tuning
custom_optimizer = Adam(learning_rate=0.0001)
model.compile(loss='categorical_crossentropy', optimizer=custom_optimizer, metrics=['accuracy'])


history = model.fit(train_images, train_labels, epochs=10, batch_size=32, verbose=0)

model.save('/content/drive/MyDrive/VGG16_custom-v1')

58889256/58889256 [==============================] - 4s 0us/step
